In [ ]:
import os
import pickle
import glob

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (13, 8)
matplotlib.rcParams.update({'font.size': 18})

from PIL import TiffImagePlugin
from PIL import Image

import rasterio
import aeronet.dataset as ds
import json
from rasterio.features import geometry_mask
from rasterio import Affine


# Resampling

Extracted TIFF images (train data) are too large to fit in the memory of most computers. Therefore, we compress and resample these images to 0.1 meters per pixel resolution. The images are also tiled (256x256 blocks). More about tiling: https://www.microscopesinternational.com/support/kb/article/ngn1266.aspx

In [ ]:
#path of tif files
train_dir = '../../data/interim/train'
train_files = glob.glob(train_dir + '/*/*/*.tif', recursive=True)
print(f"Number of images: {len(train_files)}")

#all TIFF files JPEG compressed, tiled and resampled to 0.1 m/pixel
#documentation -> https://gdal.org/programs/gdalwarp.html
for f in train_files:
    file_name = os.path.basename(f)
    directory = os.path.dirname(f)
    destination = os.path.join(directory, "res-" + file_name)
    
    print(f"Resampling {file_name}")
    command = "gdalwarp -co COMPRESS=JPEG -co TILED=YES -co NUM_THREADS=ALL_CPUS -r bilinear " + \
    f"-tr {0.1} -{0.1} {train_files[0]} {destination}"
    os.system(command)    

In [ ]:
train_dir = '../../data/interim/train'
train_files = glob.glob(train_dir + '/*/*/*.tif', recursive=True)

# Masking

For each resampled tif file create raster mask from provided geometries in .geojson file


In [ ]:
#path of resampled tif files
train_dir = '../../data/interim/train'
train_res_files = glob.glob(train_dir + '/*/*/res-*.tif', recursive=True)
print(f"Number of images: {len(train_res_files)}")

#path of geojson file for corresponding tif in the same order
geojson_files = []
#geojson_files = glob.glob(train_dir + '/*/*/*.geojson', recursive=True)
for f in train_res_files:
    parent_dir = os.path.dirname(f)
    directory, file_name = os.path.split(parent_dir)
    geojson_dir = file_name + "-labels"
    geojson_file = file_name + ".geojson"
    path = os.path.join(directory, geojson_dir, geojson_file)
    geojson_files.append(path)


In [ ]:
#returns reprojected geosjson FeatureCollection and profile of tif
def reproject(tif_prof, geojson):
    #fix geojson's coord system so that it can be reprojected
    with open(geojson, encoding="utf-8") as f:
        geojson_prof = json.load(f)
        geojson_prof["crs"] = "EPSG:4326"
    with open(geojson, "w", encoding="utf-8") as f:
        json.dump(geojson_prof, f)

    #read FeatureCollecton properties (incl. geometries) and reproject to crs as tif
    fc = ds.FeatureCollection.read(geojson)
    fc = fc.reproject(tif_prof["crs"])
    return fc

In [ ]:
#generate mask for each geojson 
for tif, geojson in zip(train_res_files, geojson_files):
    
    with rasterio.open(tif) as src:
        tif_prof = src.profile
        #print(profile)

    fc = reproject(tif_prof, geojson)

    #coords of polygons from geojson's FeatureCollection
    polygons = [f.geometry for f in fc]

    #masking all the polygons
    if len(polygons) > 0:
        mask = geometry_mask(polygons, out_shape=(tif_prof["height"], tif_prof["width"]), transform=tif_prof["transform"], invert=True).astype('uint8')
    else:
        mask = np.zeros(shape=(profile["height"], profile["width"]), dtype='uint8')

    #save masked img as tif
    destination = os.path.join(os.path.dirname(tif), "mask-" + os.path.basename(tif))
    print("Generating masks for " + os.path.basename(tif))
    profile = dict(
        driver="GTiff",
        height=mask.shape[0],
        width=mask.shape[1],
        count=1,
        crs=tif_prof["crs"],
        transform=tif_prof["transform"],
        dtype=mask.dtype,
        NBITS=1,
        )

    with rasterio.open(destination, "w", **profile) as dst:
        dst.write(mask, 1)

In [ ]:
#reproject coordinate system

tif = train_res_files[0]
geojson = geojson_files[0]
#destination = os.path.join(os.path.dirname(tif), "mask.tif")
destination = os.path.join(os.path.dirname(tif), "mask-" + os.path.basename(tif))
#print(destination)

#read tif profile
with rasterio.open(tif) as src:
    tif_prof = src.profile
    print(profile)

#fix geojson's coord system so that it can be reprojected
with open(geojson, encoding="utf-8") as f:
    geojson_prof = json.load(f)
    geojson_prof["crs"] = "EPSG:4326"
with open(geojson, "w", encoding="utf-8") as f:
    json.dump(geojson_prof, f)

#read FeatureCollecton properties (incl. geometries) and reproject to crs as tif
fc = ds.FeatureCollection.read(geojson)
fc = fc.reproject(tif_prof["crs"])

# Slicing
Slicing big tif files to small ones with size 1024x1024

In [ ]:
#path of resampled tif files
train_dir = '../../data/interim/train'
tif_files = glob.glob(train_dir + '/*/*/res-*.tif', recursive=True)

#path of corresponding masked tif files
mask_files = []
for tif in tif_files:
    directory = os.path.dirname(tif)
    mask_files.append(os.path.join(os.path.dirname(tif), "mask-" + os.path.basename(tif)))
#print(mask_files)
                       
#paths for destinations
processed_dir = '../../data/processed/train'
processed_img_dir = os.path.join(processed_dir, "images")
processed_mask_dir = os.path.join(processed_dir, "masks")

size = (1024, 1024)

In [ ]:
def sample(img, y, x, height, width):
    """
    Read sample of of band to memory with specified:
        x, y - pixel coordinates of left top corner
        width, height - spatial dimension of sample in pixels
    Return: raster, profile
    """
    #spatial coordinates of each 1024x1024 piece
    coord_x = img.transform.c + x * img.transform.a
    coord_y = img.transform.f + y * img.transform.e

    dst_crs = img.crs
    dst_name = os.path.basename(img.name)
    dst_nodata = img.nodata if img.nodata is not None else 0
    dst_transform = Affine(img.transform.a, img.transform.b, coord_x,
                           img.transform.d, img.transform.e, coord_y)

    dst_raster = img.read(window=((y, y + height), (x, x + width)),
                             boundless=True, fill_value=dst_nodata)
    return dst_raster, dict(transform=dst_transform, crs=dst_crs, nodata=dst_nodata)

In [ ]:
def generate_samples(img, width, height):
    """
    Yield samples with defined grid
    Args:
        width: dimension of sample in pixels and step along `X` axis
        height: dimension of sample in pixels and step along `Y` axis
    Returns:
        Generator object
    """
    for x in range(0, img.width, width):
        for y in range(0, img.height, height):
            yield sample(img, y, x, height, width)

In [ ]:
def save_processed(path, raster, **profile):
    """Save raster on disk"""
    c, h, w = raster.shape
    _profile = dict(
        driver="GTiff",
        height=h,
        width=w,
        count=c,
        dtype=raster.dtype,
    )
    _profile.update(profile)

    with rasterio.open(path, "w", **_profile) as dst:
        dst.write(raster)

In [ ]:
#rasterio documentation -> https://rasterio.readthedocs.io/en/latest/quickstart.html
for tif, mask in zip(tif_files, mask_files):
    city = tif.split("/")[-3]
    name = tif.split("/")[-2]
    
    i=0
    with rasterio.open(tif) as image, rasterio.open(mask) as masked:
        for sliced_image_data, sliced_mask_data in zip(generate_samples(image, *size), generate_samples(masked, *size)):
            sliced_image, image_profile = sliced_image_data
            sliced_mask, mask_profile = sliced_mask_data
            sliced_image = sliced_image[:3] #RGB channels

            if sliced_image.sum() > 100:  # prevent empty masks
                i += 1
                file_name = "{}_{}_{}.tif".format(city, name, str(i).zfill(5))
                processed_img_path = os.path.join(processed_img_dir, file_name)
                processed_mask_path = os.path.join(processed_mask_dir, file_name)

                print(f"Saving: {processed_img_path}")
                save_processed(processed_img_path, sliced_image, **image_profile)
                print(f"Saving: {processed_mask_path}")
                save_processed(processed_mask_path, sliced_mask, **mask_profile)

                print()


In [ ]:
processed_image_files = glob.glob(processed_img_dir + '/*.tif', recursive=True)
example_image = processed_image_files[10]

TiffImagePlugin.DEBUG = True

with open(example_image, 'rb') as f:
    TiffImagePlugin.TiffImageFile(f)

# Plotting the Image
img = Image.open(example_image)
plt.imshow(img)

In [ ]:
print(processed_image_files[6])
processed_mask_files[6]

In [ ]:
processed_mask_files = glob.glob(processed_mask_dir + '/*.tif', recursive=True)
example_mask = processed_mask_files[10]

TiffImagePlugin.DEBUG = True

with open(example_mask, 'rb') as f:
    TiffImagePlugin.TiffImageFile(f)

# Plotting the Image
img = Image.open(example_mask)
plt.imshow(img)